# Générateur semi-automatique de page HTML "quizz oiseaux"

Application des "Oiseaux à l'Oreille" : les quizz **2018**

In [ ]:
import os
import datetime as dt

from collections import OrderedDict as odict

import requests

from IPython.display import HTML

In [ ]:
import sys
sys.path.append('..')

import quizz

import importlib as implib

In [ ]:
KDossierQuizz = '.'

# Commutateurs de publication : locale ou sur serveur web (commenter/décommenter).
pubLocale = True

dossierSons = os.path.join(KDossierQuizz, 'enregistrements')

if pubLocale:
    
    # 1) Locale
    urlDossierSons = dossierSons.replace('\\', '/') # Jouer les sons en lisant les fichiers locaux.
    urlDossierAnnexes = os.path.join(KDossierQuizz, 'fichiers').replace('\\', '/') # idem, ailleurs

else:
    
    # 2) Internet
    urlDossierSons = '.' # Jouer depuis XenoCanto si enregistrement en vient, et n'a pas eté modifié ('-extrait')
    urlDossierAnnexes = '.' # Fichiers annexes sur mon site

In [ ]:
# Appelle de l'API Xeno-Canto avec un numéro d'enregistrement
# et affiche et renvoie qq infos à copier & coller dans le NB.
# Usage: HTML(aideEnregistrementXC(nr=416636))
def aideEnregistrementXC(nr):
    
    rep = requests.get(url='http://www.xeno-canto.org/api/2/recordings', 
                       params=dict(query='nr:{}'.format(nr)))
    rep.raise_for_status()
    dRep = rep.json()

    assert len(dRep['recordings']) == 1
    dRec = dRep['recordings'][0]

    dRec.update(alt=9999, dur=9999)
    spRec = dRec['rec'].split()
    dRec.update(id_rec='???????', ab_rec=spRec[0][0]+''.join(spRec[1:]))
    dRec.update(date_c=dRec['date'].replace('-', ''), 
                time_c=('0' if len(dRec['time']) < 5 else '') + dRec['time'].replace(':', ''))
    dRec.update(date=dt.datetime.strptime(dRec['date'], '%Y-%m-%d').strftime('%w %b %Y'))
    print("""
  lieu="{loc}", altitude={alt},
  date="{date}", heure="{time}", duree="{dur}",
  auteur="<a href=\\\"https://www.xeno-canto.org/contributor/{id_rec}\\\" target=\\\"_blank\\\">{rec}</a>",
  licence="<a href=\\\"https:{lic}\\\" target=\\\"_blank\\\">CC BY-NC-??</a>",
  fichierSon="XXIdEnreg-XXPays-{ab_rec}-{date_c}-{time_c}-XC{id}-mono-vbr5.mp3", # Nom fic. dans ./enregistrements
    """.format(**dRec))

    return 'Lien direct <a href="{url}">{url}</a>'.format(url=dRec['url'])

In [ ]:
# Les étapes : les mêmes pour tous les quizz et tous les exercices.
KEtapes = \
[quizz.DescripteurEtape(index=1, id='lancement', titre='Les enregistrements à analyser et les 1er indices'),
 quizz.DescripteurEtape(index=2, id='indices2', titre='Quelques indices à mi-parcours'),
 quizz.DescripteurEtape(index=3, id='reponse', titre='Les réponses : analyse détaillée des enregistrements')]

In [ ]:
# texte syntaxe markdown
KPreambule = \
"""
Voici donc les quiz 2018 !

Chacun d'eux, nommé comme le mois de 2018 durant lequel il a été lancé, se déroule en *3 étapes*,
publiée chacune 2 ou 3 semaines après la précédente :
1. la première, le **lancement**, vous donne accès aux enregistrements à analyser
   ainsi que quelques premiers indices pour démarrer,
2. la deuxième, à **mi-parcours**, vous donne des indices plus conséquents, pour confirmer ou pas vos premières idées
   et / ou pour aller plus loin en ciblant quelques séquences plus difficiles,
3. la troisième et dernière, pour terminer, vous donne la "**solution**", en détaillant pour chaque espèce
   les différents types de chants et cris entendus à quels moments dans chaque enregistrement.

Pour accéder à l'étape suivante, lorsqu'elle est publiée, il vous suffit de cliquer sur le **petit oeil vert**
<img height="16" src="{{dossierAttache}}/fa-eye-regular.svg" alt="Montrer"/>
situé sous le titre de cette dernière ; bien sûr, tant qu'elle n'est pas publiée, vous ne verrez rien de plus ;-)

Chaque quiz porte sur **plusieurs enregistrements** (en général 3) indépendants, pour varier les milieux et les espèces.

Pour chaque enregistrement (répété à chaque étape), vous trouverez :
* un **lecteur audio intégré**, pour l'écouter, le ré-écouter ... etc, autant de fois que nécessaire,
* un **lien de téléchargement**, pour pouvoir l'analyser tranquillement et précisément
sur votre tablette ou ordinateur, avec un lecteur plus pratique notamment en ce qui concerne
le positionnement temporel temps ; pour télécharger l'enregistrement, il vous suffit de cliquer sur le bouton
<img height="16" src="{{dossierAttache}}/fa-download-solid.svg" alt="Télécharger"/>
(à moins qu'il ne faille un clic droit, suivi de "enregistrer la cible du / le lien sous ..."),
* le nom de l'**auteur** de l'enregistrement et la **licence** de publication,
* à la dernière étape, si disponible, le lien vers la **page du site source** de l'enregistrement
  (xeno-canto.org en général).

Pub. <a href="http://audacity.fr/" target="_blank">Audacity</a> est - entre autre - un lecteur audio très pratique
et précis pour la gestion du temps : essayez-le, je vous le conseille :-) ; il est disponible gratuitement
et en toute sécurité (logiciel Open Source) sous Windows, Mac OS ou Linux (mais pas Android ou i-OS :-(.

Chaque quiz permet à chacun de jouer **quel que soit son niveau**, du débutant au plus expérimenté :
si vous ne trouvez pas tout de A à Z, c'est **normal**, certains enregistrements sont plus difficiles
que d'autres, et l'exercice n'est pas simple, lorsqu'on n'est pas immergé dans le milieu naturel ;
mais c'est l'occasion de progresser, en cherchant soi-même la solution ailleurs (les supports de formation,
bien sûr, mais aussi des CD, des applis dédiées, des livres, des sites internet
comme <a href="https://www.xeno-canto.org/" target="_blank">xeno-canto.org</a>, des amis ... etc !)

Enfin, je compte sur vous pour me les signaler les **coquilles** et **erreurs** qui se sont inévitablement
glissées dans ce qui suit, ou à me faire part de votre **désaccord** éventuel sur tel ou telle affirmation :-)
"""

In [ ]:
KAttributions = \
"""
Les photos de Fauvette à tête noire et de Mésange noire en haut de page
sont de <a href="https://www.faune-auvergne.org/index.php?m_id=7&frmAuthor=32" target="_blank">Romain Riols</a>
(tous droits réservés).
"""

## Quizz de Septembre

In [ ]:
KSeptPessadeForetConiferes = quizz.DescripteurExercice(

  index='a',
  id='PessadeForetConiferes',
  titre="Forêt de conifères à Pessade",
  lieu="Pessade, Puy-de-Dôme", altitude=1000,
  date="juillet 2018", heure="tôt le matin", duree="2 mn",
  auteur="<a href=\"https://ornithovaldallier.blogspot.com/\" target=\"_blank\">François Guélin</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr\" target=\"_blank\">CC BY-NC-SA</a>",
  milieux="""Forêt de conifères, en moyenne montagne.""", # texte syntaxe markdown
  fichierSon="ForetMoyenneMontagne-Pessade-FGuelin-201807-Matin-mono-vbr5.mp3", # nom fic. dans ./enregistrements
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict( # texte syntaxe markdown pour chaque étape
      
    lancement="""
      10 espèces au total :
      
      * dont 2 typiques du milieu naturel, et 7 plus "banales", présentes partout,
      
      * dont 3-4 au premier plan, très présentes presque tout au long de l'enregistrement,
      
      * dont 6 ont été "vues" dans la seconde moitié du module 1,
      
      * dont 1 intrus, imité par un mammifère ;-).
      """,
          
    indices2="""
      10 espèces au total :
      
      * dont 1 intrus, imité par un mammifère ...
      
        c'était une blague un peu vache, ou bovine au choix : la vache qui meugle
        vers 47s évoque bien sûr le Butor étoilé, d'où son nom latin
        <a href="http://dico-sciences-animales.cirad.fr/liste-mots.php?fiche=4385" target="_blank">
        <em>Botaurus stellaris</em></a>, à moins que ce ne soit l'inverse
        (<a href="https://www.xeno-canto.org/414619" target="_blank">exemple</a>),
        
      * d'ailleurs, avec le diptère passant en trombe près du micro vers 11.5s,
        ça pourrait même faire 11 ... 
        
        (écoutez l'effet Doppler : le son est de plus en plus grave à mesure
         qu'il s'éloigne, alors qu'il ne ralentit probablement pas ;-)

      Sinon, plus sérieusement, ... spéciale module 1 :
      
      * au 1er plan, 3 espèces avec fort volume sonore,
        toutes présentes dans le diaporama d'application du module 1 :
        
          - un "traîne-fagot" au rapport poids / puissance imbattable,
            hurlant ses trilles variées dans les aigus ;
            débuts de phrase assez isolés à 0s et 11.5s par ex.,
            
          - une espèce parfois imitatrice, très commune partout,
            dont le timbre d'une partie du chant fait souvent penser
            au Merle (mais en plus aigu) ; d'ailleurs, elle imite un peu
            en début de phrase, comme dans cette phrase quasi-typique
            et isolée qui commence à 1mn04s,
            
          - une espèce de grand turdidé (toi-même ;-) plus calme et posé ;
            2 débuts de belles phrases à 8.5s et 11.5s
            (bel résonance sous les grands arbres !),
            
      * au 1er plan toujours, mais plus discret, un autre "petit agité",
        plus constant dans les hauteurs aiguës, mais bien moins bruyant
        et près du sol, au chant plus délicat ;
        
        nicheur typique des forêts de montagne, alors que son cousin
        presque jumeau se trouve plutôt en forêt de plaine ;
        
        débuts de phrase isolés à 7s et 31s par exemple,
        
      * à l'arrière plan, 4 autres espèces plus difficiles à saisir,
        car elles sont loin et c'est "un peu" mélangé tout ça ; les 3 1ères
        dans le diaporama d'application du module 1 ; par ordre
        de difficulté croissante :
        
          - un fringille très commun presque partout, au chant assez stéréotypé,
            puissant, sonore et portant loin ; 2 phrases à 25-26s, 53-54s,
            
          - un petit turdidé très commun, des milieux frais ;
            1 seul début de phrase isolé, à 1mn16.5s,
            
          - un petit insectivore discret et peu farouche, très commun,
            au chant aigu et rythmé bien familier ... qui surnage difficilement
            sur le fond sonore bien chargé entre 1mn44s et 1mn49s,
            avec 3 notes audibles dans un silence relatif d'1s vers 1mn46.5s,
            
          - une mésange au chant rythmé, attention à la confusion,
            avec 1 fin de phrase de 3 notes bien lointaines mais assez "seules",
            entre 1mn34s et 1mn35.5s (spéciale module 2)
            
      * et la 9ème, c'est une espèce émettrice de "hou" graves multiples,
        mais qui ne fait pas peur aux passereaux ; à la fin de l'enregistrement ...
      """,
          
    reponse="""
      Les "braillards" du premier plan :
      
      * Merle noir (chant) : 5.5s, 8.5s, 14s, 20.5s, 30.5s, 37s, ... etc,
      
      * Fauvette à tête noire (chant) : 1s, 3.5s, 14s, 26s, 34.5s, 44.5, 55s ... etc,
      
        Anecdote (spéciale niveau 3) : à 64s, début de phrase "façon" Fauvette des jardins
          (moins grinçant, plus liquide et bien mieux rythmé) !
        
      * Troglodyte mignon (chant) : chant : 0-1s (fin de phrase), 10.5-14.5s, 22.5-28s,
            37s (bribe interrompue), ~43s-46s, ~56s-1mn00s, ~1mn08s-1mn13s, ~1mn18s-1mn22s,
            ~1mn39s-1mn43s, ~1mn49s-1mn53, ~2mn01s-2mn04s,
            
      * Roitelet huppé (1er plan, braillant aussi à sa manière ténue et très aiguë) :
      
          - chant : 1.2, ~7, ~15, ~21, ~30 et ~40s,
          
          - cris : une série à 47s ... ensuite, on n'entend plus que des minuscules "si"
            courts et plutôt isolés, avec 1 plus fort et vibré, bien audible à 57.5s.

      A l'arrière plan, plus loin, plus difficile :
      
      * Rougegorge familier (loin) : chant, au moins vers 7-8s, 18s, 29s, 41s, 61s, 1mn16.5s, 1mn48ss
         (mais pas facile, noyé dans le fatras),
         
      * Pinson des arbres (loin) : chant 25-26s, 53-54s,
      
      * Mésange noire (loin) :
      
          - chant : 1mn33.8s-1mn35.6s,
          
          - cris typiques à l'unité, modulés, fins, lointains :
            18.7s, 20.7s (2 cris différents), 50s, 52s, 52.5s, 54s, 1mn01s, 1mn07.3s,
            
      * Pigeon ramier : chant bien audible, à la toute fin, 1mn48.5s,
      
      * Pouillot véloce (loin) : chant bien rythmé (notre chance dans ce fatras) à 1mn44.3-1mn49s.
      """))

HTML(KSeptPessadeForetConiferes.lecteurHtml(urlDossierSons))

In [ ]:
KSeptVienneForetPlaine = quizz.DescripteurExercice(

  index='b',
  id='ForetPlaineVienne',
  titre="Forêt feuillue de plaine, dans la Vienne",
  lieu="Forêt domaniale de Vouillé-St-Hilaire, Quincay, Vienne", altitude=150,
  date="mars 2013", heure="7 heure du matin", duree="1 mn",
  auteur="<a href=\"https://www.xeno-canto.org/contributor/WXEEFPSPJM\" target=\"_blank\">Jack Berteau</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/3.0/\" target=\"_blank\">CC BY-NC-SA</a>",
  milieux="""Chênaie-charmaie en plaine.""", # texte syntaxe markdown
  fichierSon="ForetPlaine-Vienne-JBerteau-20130309-0700-XC155978-mono-vbr5.mp3", # nom fic. dans ./enregistrements
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict( # texte syntaxe markdown pour chaque étape
      
    lancement="""
      * 3 espèces de pics (l'une pas facile, juste 2-3 cris discrets),
      
      * 2 de fringilles communs (l'un nettement plus que l'autre quand même),
      
      * et au moins 3 autres chanteurs, très communs.
      """,
          
    indices2="""
      * 3 espèces de pics (spéciale module 3) :
      
          - le chanteur, vous l'avez déjà trouvé, pas d’ambiguïté,
          
          - le tambourineur ... il peut y avoir confusion entre 2 espèces
            sur le rythme et la longueur, mais pas sur la puissance sonore :-),
            
          - le discret, mais vraiment discret (1 cri à 22.5s, et 2 enchaînés
            à 24.0s) ... très dur : c'est le cas le plus difficile sur le terrain,
            où l'oiseau n'émet que le début d'une série typique, et s'arrête ;
            du coup, ça ressemble à l'autre espèce, voisine physiquement !
            
      * 2 de fringilles communs (spéciale module 3) :
      
          - l'une déjà entendue à Pessade, très commune presque partout, en arrière plan,
            phrases sonores à 5s, 10s, 17s par ex.,
            
          - l'autre beaucoup plus difficile ici (et ailleurs), avec uniquement des cris, de 2 types :
          
              * courts et sur-aigus : 1er type à 33.5s, 35.5s, 35.9s,
              
              * 2nd type à 50.3s, 50.7s, 50.0s, 51.4s ;
               
            une espèce discrète passant souvent inaperçue, sauf aux mangeoires !
           
      * et au moins 3 autres chanteurs, très communs :
      
          - 1 "gros farouche" que vous avez déjà repéré, forcément,
          
          - et 1 "petit hargneux" pas loin du 1er plan, déjà entendu à Pessade,
            un peu noyé dans le brouhaha, mais dont 1 type de trille
            bien sonore ressort souvent, par ex. à 2s, 7s, 14s, 23s ... etc (Cf. module 1),
            
          - 1 mésange au chant rythmé, comme à Pessade, mais ...
            2 phrases à la fin de l'enregistrement, commençant à 47s et 53s,
            assez loin, pas facile ; des motifs à 4 notes pas très courants,
            mais avec des notes vibrées typiques (Cf. module 1),
           
      * et un cri assez long vers 7s, bien audible au loin, assez indistinct,
        j'hésite entre plusieurs espèces et une autre qui les imite souvent ...
      """,
          
    reponse="""
      * 3 espèces de pics :
      
          - Pic noir (tambour) : puissant, très sonore, dans les graves,
            et surtout très long, sur un rythme assez lent, sans hésitation,
            et s'atténuant bien progressivement à la fin : 1-3s, 16.5-18.5s, 44-45s, 
          
          - Pic vert (chant) : 1 claire et unique phrase, très classique, à 18-23s,
          
          - Pic mar (pas facile, juste des cris discrets : 1 vers 22.5s, et 2 enchaînés vers 24s),
          
      * 2 espèces de fringilles communs :
      
          - Pinson des arbres (chant) : à environ 5s, 10s, 17s, 21s, 26s, 30s, 38s, 45s, 55s,
          
          - Grosbec casse-noyaux : uniquement des cris, de 2 types :
          
              * puissants, courts et sur-aigus, explosifs :
                des "kix", à 33.5s, 35.5s, 35.9s,
                
              * plus longs, moins puissants, sur-aigus aussi :
                des "psiî" à l'envol, à 50.3s, 50.7s, 50.0s, 51.4s,
          
      * et au moins 3 autres chanteurs, très communs :
      
          - Troglodyte (chant) : au deuxième plan, phrases typiques complètes (de 4-5s),
             à environ 1s, 6.5s, 11s, 22s, 29s, 37s, 46s, 52s (au moins) ;
             
             par moment, probablement 2 individus proches démarrant l'un sur la fin de phrase de l'autre ...
          
          - Pigeon ramier (chant) :
          
              * 1 individu à 0s et 36s,
              
              * 1 autre, plus loin, à 55s (à moins que ce ne soit le même qui se serait éloigné),
              
              * peut-être 1 autre, à peine audible très très loin, à 6s, 9s, 28.5s au moins ...
              
                mais c'est peut-être aussi un "mirage sonore" dû au bruit d'enregistrement
                et à celui de l'avion qui passe au loin ...
          
          - Mésange charbonnière (chant) : 2 phrases à la fin, à 47-50s et 53-56s,
          
      * et 2 cris différents, de rapace, ou d'un geai l'imitant:
      
          - comme un sifflement ou un miaulement descendant à 6.7-7.8s :
            Milan noir ? Buse variable ? autre chose ? mystère ...
          
          - un cri plus rauque, court, sonore, comme s'évanouissant progressivement dans la forêt :
            Buse variable ? Autour des palombes ? autre chose ? mystère encore ...
      """))

HTML(KSeptVienneForetPlaine.lecteurHtml(urlDossierSons))

In [ ]:
KSeptBerlinVille = quizz.DescripteurExercice(

  index='c',
  id='VilleBerlin',
  titre="En ville, à Berlin",
  lieu="Quartier de Rummelsburg, Berlin, Allemagne", altitude=30,
  date="11 mars 2017", heure="7 heure du matin", duree="30 s",
  auteur="<a href=\"https://www.xeno-canto.org/contributor/ZGQBFCIWJI\" target=\"_blank\">Annette Hamann</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/4.0/\" target=\"_blank\">CC BY-NC-SA</a>",
  milieux="""En pleine ville, pas loin d'un jardin public (ou d'un espace vert ?).""", # texte syntaxe markdown
  fichierSon="Ville-Berlin-AHamann-20170311-0700-XC358731-mono-vbr5.mp3", # nom fic. dans ./enregistrements
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict( # texte syntaxe markdown pour chaque étape
      
    lancement="""
      * 3 espèces de fringilles très communs, dont 1 très difficile à isoler,
      
      * 1 espèce très commune partout autour de nous, peu discrète.
      """,
          
    indices2="""
      * 2 espèces de fringilles très communes (spéciale module 3):
      
          - la 1ère occupe presque tout l'enregistrement, du début à la fin,
            avec 2 phrases continues, séparée par un silence entre 14 et 17s ;
            
            un flot sur-aigu et sur-excité, où il est impossible de séparer
            les notes tellement elles sont rapides et courtes,
            
          - la seconde n'émet que 2 notes prolongées, peu sonores, peu "tonales"
            ou mélodiques (mais pas totalement rêches non plus),
            1 dans chaque phrase de l'espèce précédente,
            
        NB : Ces 2 espèces, pourtant d'aspect assez différent, ont la même
             manière de chanter en paradant.
             
       * 1 espèce au chant plus grave,
         qu'on entend aussi dans la forêt de moyenne montagne à Pessade,
         mais vous l'avez déjà trouvée ... (module 1),
         
       * et puis une 3ème espèce de fringille, très commune dans nos jardins,
         très difficile à percevoir ici, dont seules les notes les plus "claquantes"
         surnagent du chant continu de la 1ère espèce au début de sa 2nde phrase
         (et même un peu avant, à faible volume), mais là, c'est vraiment difficile (module 3),
         
       * et pour le "diûu" bizarre au timbre de Merle à 11s, 12s, 18s, 27s, 29s ... mystère ...
      """,
          
    reponse="""
      * 3 espèces de fringilles très communs :
      
          - Serin cini : 2 phrases longues classiques : 0s et 17s,
          
          - Verdier d'Europe : zzz électrique à 5.5s et 26s,
          
          - Chardonneret : commence un peu avant 16s, avant la reprise du Serin à 17s,
            et chante en même temps que ce dernier ;
            
            difficile à isoler, mais les cliquetis surnagent au dessus de la bouillie du Cini,
            jusqu'à la fin semble-t-il, avec qq moments plus discrets :-).
            
            Comparer :
            
                * la fin de la 1ère phrase du Cini (entre 10 et 14s par ex.),
                
                * le début de la suivante (entre 17 et 21s par ex.),
                
            => il y a des "choses" en plus dans la 2nde version ... 
               qui commencent d'ailleurs discrètement vers 16s ...
               
      * 1 espèce très commune autour de nous, peu discrète : le Merle noir (chant) : 
      
          - 1 au premier plan : 0s (fin de phrase), 6.5s, 15.5s, 26.5s,
          
          - 1 (au moins) à 'arrière plan : ... 19.5s, 22.5s, (25.5s)
              
      * ~~un "diûu" bizarre que je n'identifie pas (timbre de Merle, mais ...) à 11, 12, 18, 27, 29s,~~
      
      * ~~et l'auteur de l'enregistrement signale aussi un geai des chênes, mais je ne l'entends pas :
        qui le trouvera ?~~
        
      * et le "diûu" bizarre au timbre de Merle que je n'ai pas identifié, à 11, 12, 18, 27, 29s,
        eh bien c'est le geai signalé par l'auteure de l'enregistrement ... en fait de "diuû",
        avec la distance, ce doit être un dérivé du *"triheuuu" éraillé à finale un peu vibrante*,        
        un "classique" parmis les infinies possibilités de "simagrées" de l'espèce, particulièrement
        bruyante et inventive au début du printemps, lorsque plusieurs jeunes individus (5, 8, 12 ...)
        traversent peu discrètement la campagne et les bois en se poursuivant en tous sens
        et en paradant à qui mieux mieux (une sorte de "lekking itinérant" en période de formation des couples).
        
        Exemples (sans ce "diuû exactement, mais avec ce "triheuuu" vibrant au moins dans les 2 derniers,
        et plein d'autres choses, dont des imitations) :
          * <a href="https://www.xeno-canto.org/309173" target="_blank">en forêt en France</a>,
          * <a href="https://www.xeno-canto.org/412473" target="_blank">en forêt de montagne en Espagne</a>,
          * <a href="https://www.xeno-canto.org/254581" target="_blank">en ville en Angleterre</a> ...
        
        D'ailleurs, la répétition irrégulière du même motif (à qq secondes d'intervalle, par-ci par-là)
        est assez caractéristique de l'espèce : comme un "riff" de guitare qu'on aime bien :-)
        
        Merci à Thibault Brugerolle pour l'identification et le rappel
        du *"triheuuu" éraillé à finale un peu vibrante* ... bon sang, mais c'est bien sûr !
        
      * quand au double claquement mat, au loin, avec un peu d'écho, à 3.7 et 3.9s,
        je ne sais pas ce que c'est ... encore le geai ? ou plutôt un bruit anthropique ?
      """))

HTML(KSeptBerlinVille.lecteurHtml(urlDossierSons))

In [ ]:
KSeptConocephaleGracieux = quizz.DescripteurAnecdote(
  index='a',
  id='ConocephaleGracieux',
  titre="Tellement aigu ...",
  auteur="<a href=\"http://jpmeuret.free.fr/nature.html\" target=\"_blank\">Jean-Philippe Meuret</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr\" target=\"_blank\">CC BY-NC-SA</a>",
  fichierSon="20180820-2335-StAgoulinEglise-JPM-ConocephaleGracieux.mp3", # nom fic. dans ./enregistrements
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
  texte="""
    Et pour l'anecdote, hors quiz, certain(e)s connaissent déjà : pas un oiseau, mais une sauterelle,
    enregistrée dans mon jardin fin août (elle a chanté jusqu'à la mi-septembre),
    que je n'avais jamais vue ou entendue avant, une découverte ...
    
    Attention (sérieusement) pour les oreilles jeunes : pour l'écouter, baisser le volume
    à 10 %, et si vous utilisez un casque, écarter les écouteurs des oreilles au début ;
    ensuite, augmenter progressivement le volume.
    Ce n'est pas dangereux, mais ça peut surprendre ... 
    
    C'est le <a href=\"http://www.naturemp.org/Conocephale-gracieux.html\" target=\"_blank\">
    Conocéphale gracieux (<em>Ruspolia nitidula</em>)</a> ;
    à la stridulation continue, sur-aiguë, et sur-puissante, insupportable paraît-il
    pour les très jeunes oreilles (ma fille de 14 ans confirme !), mais dont je n'entends
    que la partie basse du spectre (et sans doute à peine 5 % de la puissance
    sonore - ce qui me la rend très supportable - puisque je n'entends
    plus rien au dessus de 13500 Hz, ce qui est normal à 50 ans passés).
    
    Le spectre émis est très large (du très aigu aux ultra-sons),
    et si l'essentiel de la puissance sonore s'exprime entre 13000 et 18500 Hz,
    les "hostilités" commencent déjà à 9500 Hz (au dessus des roitelets,
    et autres grimpereaux, pourtant déjà très aigus !).
    
    J'en suis désolé, mais certain(e)s d'entre vous risquent bien de ne pas
    entendre grand chose, même à fort volume ... âge oblige.
    
    Sans doute une espèce qu'on est amené à entendre de plus en plus souvent,
    au moins en plaine, par les temps de réchauffement qui courent.
    """)

HTML(KSeptConocephaleGracieux.lecteurHtml(urlDossierSons))

## Quizz de Novembre

In [ ]:
KNovCoupeForetVienne = quizz.DescripteurExercice(

  index='a',
  id='CoupeForetVienne',
  titre="Coupe forestière dans la Vienne",
  milieux="""Parcelle en régénération, quelques années après une coupe à blanc.""", # texte syntaxe markdown
  lieu="Forêt domaniale de Vouillé-St-Hilaire, Quinçay, Vienne", altitude=130,
  date="4 juin 2018", heure="17h", duree="49 s",
  auteur="<a href=\"https://www.xeno-canto.org/contributor/WXEEFPSPJM\" target=\"_blank\">Jack Berteau</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/4.0/\" target=\"_blank\">CC BY-NC-SA</a>",
  fichierSon="CoupeForet-Vienne-JBerteau-20180604-1700-XC438418-mono-vbr5.mp3", # nom fic. dans ./enregistrements
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict( # texte syntaxe markdown pour chaque étape
      
    lancement="""
      Une scène assez aérée, par le peu d'artistes présents, ... mais aussi par le vent !
      
      4 espèces communes donc, chantant toutes, dont :
      * 1 fringillidé,
      * 1 turdidé,
      * 2 sylvidés.
      """,
          
    indices2="""
    """,
          
    reponse="""
      Au premier plan, on l'imagine perchée à quelques mètres du micro, au faîte d'un buisson ou d'un jeune arbre,
      à demi dissimulée par les feuilles de ses rameaux terminaux :
      
      * Linotte mélodieuse : phrases typiques du chant, relativement courtes ici, assez calmes
        (noter le côté moins mélodique, plus "nasal", et moins aigu, de la plupart des notes ici,
         à comparer au Chardonneret du Montado portugais ci-dessous),
        
      Au deuxième plan, un peu plus loin :
      
      * Pouillot véloce (chant) : phrases un peu hésitantes par moment (assez courant),
      
        commencent à ~0-3s, 7-14s, 20-28s, ... etc ; 
      
        à noter : les petites notes double "têdêt" moins tonales / mélodiques, moins sonores, assez mates,
        qui précèdent assez classiquement les phrases typiques "tip tip ..." bien scandées
        (peut-être intriguant lorsqu'elle ne sont pas suivies "d'effets" ... ce qui arrive) ;
        
        à ~4.5-6.5s, 16.5-18.5s, 28.5-29.5s, 30.5-31.5s,
        
      * Merle noir (chant) : ses notes sonores résonnent légèrement sur une lisière assez proche,
        lorsque le vent le permet ; au moins 1 autre chanteur dans le secteur (audible furtivement à ?? s et ??s).
      
      * Fauvette à tête noire (chant, à la toute fin, 44-45s) : 
        un peu plus éloignée semble-t-il, à moins que ce ne soit le vent qui brouille nos repères ...
    """))

HTML(KNovCoupeForetVienne.lecteurHtml(urlDossierSons))

In [ ]:
KNovBordEtangPologne = quizz.DescripteurExercice(

  index='b',
  id='BordEtangPologne',
  titre="Bord d'étang en Pologne",
  milieux="""Lisière de forêt au bord d'un étang.""", # texte syntaxe markdown
  lieu="Comté de Legionowo, Province de Masovia, Pologne", altitude=80,
  date="4 mai 2013", heure="9h30", duree="3 mn 55 s",
  auteur="<a href=\"https://www.xeno-canto.org/contributor/ZNCDXTUOFL\" target=\"_blank\">Jarek Matusiak</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/3.0/\" target=\"_blank\">CC BY-NC-SA</a>",
  fichierSon="BordEtang-Pologne-JMatusiak-20130504-0930-XC133056-mono-vbr5.mp3", # nom fic. dans ./enregistrements
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict( # texte syntaxe markdown pour chaque étape
      
    lancement="""
      Pour ce quiz, on se limitera à la 1ère minute de l'enregistrement
      (mais rien n'interdit d'aller plus loin pour le plaisir d'une espèce supplémentaire ...
       et demie, et d'une chose bizarre ...).
      
      Dans cette 1ère minute donc, en plus des grenouilles vertes / rieuses, et de quelques
      chuchottements humains (mais ... on dirait du Français ?), 4 espèces bien présentes, dont :
      * 2 picidés,
      * 1 turdidé,
      * 1 fringille commun.
      
      Et peut-être en bonus, un sylvidé, très loin sans certitude ...
      """,
          
    indices2="""
    """,
          
    reponse="""
      Au premier plan :
      
      * Pic mar (chant) : séries lentes de "kweï" (1 / 2s) éraillés, déchirants, traînants ...
        évoquant un râle assez aigu, un cri de douleur ou de détresse animale, 
        mais avec des harmoniques plus graves,
        
        à 5s, 17s, 34s
      
      Au second plan :
      
      * Torcol fourmilier (chant) : séries assez rapides de "kweï" (2-3 / s) plus aigus,
        pas traînants, plus courts que ceux du pic mar,
        
        à 1s, 11s, 20s, 30s, 36s, 46s, 56s,
        
        rythme assez similaire au Pic noir, c'est vrai,
        mais ici, c'est plus aigu, plus grinçant, et il n'y a pas les harmoniques
        plus graves du Pic noir, ce n'est pas le même gabarit !
        (comparer avec http://jpmeuret.free.fr/chants/pics-2018/pics-2018.html#PicNoir.ch)
        
      * Rossignol philomèle (chant) : 0s, 4s, 9s, 14s, 22s, 32s, 41s, 46s, 51s, 55s, 
      
      * Pinson des arbres :
      
          * chant : phrases complètes à ~10s, 19s, 27s, 37s, 52s ;
          
            noter le "kik" terminal de toutes les phrases, après la séquence habituelle en Auvergne
            (parfois bien audible comme à 21s et 55s, mais souvent très étouffé ici), variation dialectale
             que l'on peut aussi entendre dans le sud de la France, et de l'Auvergne
             (mais pas dans le Puy-de-Dôme à ma connaissance ... peut-être au sud ?),
            
          * cris : un "tiug" de vol à 42s (c'est un autre individu, puisque le chanteur continue après).
          
      Loin à l'arrière plan,
      
      * peut-être une Fauvette à tête noire, vers 45s (bribe de phrase flûtée).
            
      Les grenouilles vertes / rieuses (je ne sais pas les distinguer) coassent à 2 hauteurs distinctes,
      qui correspondent probablement à 2 types de chants (fonctions différentes ?) :
      
      * la plupart du temps, hauteur "normale", "cris" longs et traînants, tout au long de la séquence,
      
      * par moment un plus bas, plus étouffé, plus résonnant, des cris plus courts, comme vers 26s, 36s.
          
      Pour les gourmand(e)s, au delà de la 1ère minute :
      
      * Pic vert : quelques débuts de phrases de chant, par très "affirmées" (légère inquiétude,
        due à la présence de l'espion au micro ?),
        
        à 3mn06s et 3mn13s,
      
      * Loriot d'Europe : possible bribe à 1mn40s,
      
      * Pic mar : les phrases de chant se poursuivent, mais tournent, à la fin (après 2mn10s),
        à des râles plus graves, plus traînants, désespérés, épuisés ... c'est l'agonie ;-),
        
        ex. à 2mn12-2mn15s, 2mn29s, 2mn38s, 2mn46.5s-2mn49s, ... etc,
        
        chez les espèces qui ont des cris très traînants, il peut
        y avoir des variations importantes de ce genre : les cordes vocales
        à peu près dans la même position, mais on pousse moins et "variablement" la voix
        (ex: chez le Geai, le cri habituel peut être plus grave et moins sonore)
        
      * Pinson des arbres : les phrases sont souvent tronquées à la fin.
    """))

HTML(KNovBordEtangPologne.lecteurHtml(urlDossierSons))

In [ ]:
KNovMontadoVignesPortugal = quizz.DescripteurExercice(

  index='c',
  id='MontadoVignesPortugal',
  titre="Montado et vignes dans l'Alentejo au Portugal",
  milieux="""
    <a href="https://fr.wikipedia.org/wiki/Dehesa" target="_blank">Montado</a>
    (de chênes lièges ?), vergers ? vignes ? (on "y" voit pas bien de si haut).""", # texte syntaxe markdown
  lieu="Estremoz, dans l'Alentejo Central, au Portugal", altitude=420,
  date="5 mai 2018", heure="7h30", duree="36 s",
  auteur="<a href=\"https://www.xeno-canto.org/contributor/OOECIWCSWV\" target=\"_blank\">Peter Boesman</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-nd/4.0/\" target=\"_blank\">CC BY-NC-ND</a>",
  fichierSon="MontadoVignes-Portugal-PBoesman-20180505-0730-XC416636-mono-vbr5.mp3", # nom fic. dans ./enregistrements
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict( # texte syntaxe markdown pour chaque étape
      
    lancement="""
      Un enregistrement assez abimé par un nettoyage un peu violent :-(
      
      Mais on y entend 2 espèces de fringilles communs :
      
      * l'une clairement, par son chant répété au premier plan,
      
      * la seconde plus "furtive", par quelques cris typiques, au deuxième plan,
        entre les phrases de la première espèce.
      
      Probalement aussi 2 autres espèces très commensales de l'homme en arrière plan,
      mais pas faciles à reconnaître.
      """,
          
    indices2="""
      * pas un pinson, qui fait des "diuc" ou "diug", mais plutôt plus isolés,
        par en séries rapides de 2-3 (sauf dans une bande, c'est vrai) ;
      * pas un verdier fait des "tududu" encore plus proches ;
        ici, c'est plus "tiog" ... avec un côté liquide ...
      * mais c'est vrai qu'il y a de quoi tourner chèvre ...

    """,
          
    reponse="""
      Les 2 espèces que l'on entend assez clairement :
      
      * Chardonneret (premier plan, oiseau posé) : chant assez typique, à phrases de longueur moyenne,
        plutôt stéréotypées ici (individu peu inventif ?), souvent terminée par une crécelle sèche "zrrrrz" ;
        
        mais cet individu a un chant tout de même moins claquant qu'à l'habitude (variation géographique
        ou individuelle ? mystère ...), plus doux ;
        
        par rapport à la linotte de la coupe forestière ci-dessus, notes plus tonales et plus aiguës,
        plus nettes et moins nasales ; enchaînement des notes moins rapide aussi ;
        
        débuts de phrase vers 1s, 4s, 8s, 12s, 16.5s, 24s, 26.5s, 31s, 34s.
        
      * Linotte mélodieuse (deuxième plan) : séries typiques de 2-4 cris assez gutturaux ou nasaux,
        et un peu liquides "tiok tiok ..." enchaînés très rapidement, très souvent émises en vol,
        mais pas ici semble-t-il ;
        
        discrets, vers 3.2s, 7.7s, 14.3s, ~23s, ~29s au moins.
        
      Et peut-être, en arrière plan (le "nettoyage" de l'enregistrement rend l'identification incertaine) :
      
      * Moineau domestique : par moment, comme la rumeur des multiples cris d'un groupe assemblé dans un buisson ...
      
      * Merle noir juvénile quémandant ? à 3.5s, 5.5s, 8s, 11s, 13s, 16s, .. etc,
        des "tsi tsi tsi" qui pourraient y faire penser.
        
      Et un grincement bizarre à 22.5-23s, en 2 temps, un long et un court ...
      qui pourrait faire penser à un geai, un héron, une perdrix ... ou une pintade ... ou ...
      
      Difficile de décider, c'est trop indistinct, mais ...
      * les geais font effectivement ce type de grincement double,
      * les cris de héron sont presque toujours accompagnés d'un écho lointain,
        le temps que son cri puissant et à composantes graves revienne des obstacles forcément éloignés
        (les hérons préfèrent le terrain découvert, même en l'air), ... et ici, pas d'écho lointain ...
      * l'hypothèse "basse-cour" n'est pas idiote, avec la présence probable des moineaux ...
      
      Mystère ...
    """))

HTML(KNovMontadoVignesPortugal.lecteurHtml(urlDossierSons))

In [ ]:
KNovClairiereArdenneBelge = quizz.DescripteurExercice(

  index='d',
  id='ClairiereArdenneBelge',
  titre="Clairière en Ardenne belge",
  milieux="""Clairière à grands arbres, prairies, friches, habitations.""", # texte syntaxe markdown
  lieu="Saint-Hubert, province du Luxembourg en Wallonie, Belgique", altitude=360,
  date="26 janvier 2018", heure="9h40", duree="1 mn 46 s",
  auteur="<a href=\"https://www.xeno-canto.org/contributor/PZOLZQBBWM\" target=\"_blank\">Daan Drukker</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/4.0/\" target=\"_blank\">CC BY-NC-SA</a>",
  fichierSon="ClairiereArdenne-Belgique-DDrukker-20180126-0937-XC401097-mono-vbr5.mp3", # nom fic. dans ./enregistrements
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
    
  etapes=dict( # texte syntaxe markdown pour chaque étape
      
    lancement="""
      Et pour finir en beauté, une belle ambiance avec de la profondeur :
      imaginez-vous sous des grands arbres, en lisière d'une zone de friche et de prairies,
      avec une haute lisière à 600 ou 800m ...
      
      Et il y a du monde :
      
      * 4 espèces qui se font bien entendre, au premier plan, ou avec une bonne voix :
          - 2 paridés, dont un invité de marque, une espèce pas rare, mais assez localisée en Auvergne, 
          - 2 corvidés,
          
      * 6 à 8 espèces en plus, souvent lointaines, et / ou ne se manifestant que par des cris,
        parfois très furtifs : 
          - 1 ou 2 autres mésanges,
          - 1 pic,
          - 1 fringille,
          - 2 turdidés,
          - 2 passereaux typiquement arboricoles,
          - 1 possible gros "non-passereau".
      """,
          
    indices2="""
    """,
          
    reponse="""
      Au premier plan, l'invitée d'honneur et une cousine proche :
      
      * Mésange boréale :
      
          - 2 types de chant :
              * l'habituelle et typique série de 3-4 "tsiû" descendants,
              
                à 1s, 5s, 14s, 19s, 26s, 
                
                à ne pas confondre avec le chant très similaire de la Sitelle torchepot,
                mais qui fait plutôt "tiuû" ... ah flûte, c'est pareil ...
                mais avec une sonorité plus ronde, une composante plus grave
                (qui va bien avec cet oiseau plus robuste que la boréale) ...
                
                =\> lien vers exemple ?????
                
              * une version moins courante (jamais entendue pour moi) "ti ti tu tuîuuu"
                à la note finale traînante, vibrée, au timbre électronique étrange,
              
                à 33.5s, 39s, 48s, 54s, 1mn25s, 1mn29s, 1mn34s, 1mn36.5, 1mn40.5s, 1mn44s,
              
          - et les cris nasillards et traînants "tiaân" typiques (ici en séries de 3, très courantes),
          
            à 1mn08s, 1mn13s, 1mn16.5s, 1mn21.5s, 
          
      * Mésange bleue :
      
          - le plus souvent des cris :
          
              * des séries descendantes de "tsi" ou "si" discrets :
                à 1s, 5s, 19s, 32s, 35s, 38.5s, 42.5s, 46.5s, 52.5s, 56.5s ... etc,
                
              * la roulade / crécelle roulée un peu nasillarde en "é" : 
                à 8.5s, 36s, 1ms02s, 1mn10.5s, 
          
          - des phrases du chant typique "tsi tsi dédédé..." : à 1mn02s, 1mn09s, 1mn14s, 1mn20s, ... etc,
        
      Au deuxième plan, 2 autres espèces bien audibles aussi :
      
      * Pie bavarde : des cris excités courts, doubles ou triples, chuintés, "tchia-tchiak", presque sans voyelle,
        à 2s, 4s, 7s, 10s, 14s, 17s, ... etc,
      
      * Corneille noire : croassements habituels, à 15s, 47s, 1mn02s, 1mn43s,
      
      Et 6 à 8 autres plus diffiles, voire incertaines, par furtivité ou grand éloignement :
      
      * Moineau domestique : cris d'un individu au moins (mais pas nombreux ?) stationnant pas très loin,
       à 2s, 4s, 10-13s, et toutes les 1, 2 ou 3 s entre 16s et 32s.
        
      * Mésange nonnette (assez "certaine") : assez loin, un "tiou rin rin rin" ("tioû" explosif) à 50.6s
        (à moins que ce ne sois une charbonnière imitatrice ... très courante) ;
        
        d'autres cris type Nonnette "p-tiu" très courts à 1mn12-1mn13s (mais faussaire Charbonnière pas exclue),
      
      * Grive draine (assez "certaine") : lointaine mais claire, à 14.5s, moins claire mais très probable aussi
        à 4.5s, 8.5s, 11.5s, 21.7s, 24.5s, 30.5s, 53.5s, 57s (au moins),
        
      * Merle noir : à 7.4-7.8s, un "siii" très aigu, bien appuyé, et vibré, cris émis par des individus
        plutôt calmes semble-t-il, ou en très légère inquiétude, à moins que ce ne soit de l'agacement
        (fonction inconnue pour mo en fait :-) ... proche des "siii" prolongés du Grimpereau des jardins,
        (même fréquence d'~7500Hz) mais moins véhément ;
        
        le sonagramme montre 2 "raies" spectrales distinctes, déjà observées sur un autre enregistrement
        de Merle avéré, alors que sur 3 enregistrements de xeno-canto.org, les cris attribués
        par l'auteur au Grimpereau des jardins n'ont qu'une seule raie (entre 6500 et 7500 Hz) ...
        
        ??? =\> montrer les sonagrammes ????
        
      * Pic épeiche (très probable) : même si un seul "kik" isolé (de 4 100tème de s),
        même assez clair et pas trop éloigné, ça peut paraître mince : à ~52s,
        
        je vous jure, sur le sonagramme, en zoomant bien, il n'y a pas de confusion possible
        (même pas avec un Pic mar) !
        
        =\> ???? montrer le sonagramme, avec celui d'un Mar ????? 

      * Grimpereau des jardins (signalé par l'auteur) : les "sii" vibrés à 11.7s et 13.4s
        sont probablement émis par un (au moins) grimpereau
        (d'ailleurs, le sonagramme ne montre qu'une seule raie spectrale ...
         et à 8500Hz, contre 2 raies et 7500Hz pour le Merle) ;
         
        et puis difficile d'en être certain, avec toutes ces mésanges capables elles aussi de divers "si"
        si courts (;-), mais les 2 "si" courts à 12.1s, 12.5s, 16.1s, 17.5s, 21.5s, ... etc,
        sont probablement aussi d'un grimpereau des jardins,
        
        bon, ça pourrait aussi être un Grimpereau des bois, c'est vrai (je ne sais pas les distinguer
        sauf au chant, et encore, ils s'imitent) ... d'autant que les 2 espèces sont nicheuses
        dans les Ardennes belges ... mais faisons confiance à l'auteur :-)
      
      * Pinson des arbres (probable) : 1 unique cri "tuing" d'inquiétude / alarme à 0.3s
        (mais pour être certain de l'identification, il faudrait quelques répétitions ...),
      
      * Pigeon ramier (incertain) : 2 séries de 3 "hou" très lointains, à 57.7s, 1mn05s ;
      
        vous me direz : "3 hou", c'est pas pigeon, c'est tourterelle (turque) ... je sais, mais ici,
        le rythme des 3 "hou" ressemble à celui des 3 1ers du Ramier : hou hououou hou (le 2ème prolongé),
        alors que chez la Turque, c'est entre le 2nd et le 3ème qu'il y a un plus grand silence ...
        
        bon d'accord : Ramier ou Tourterelle turque ... ou autre chose ... mystère ...
      
      A 1mn25s, enfin, il se produit un changement de volume / de bruit dans l'enregistrement,
      sans doute un filtrage différent de ce qui a été appliqué avant ;
      dans cette partie, on entend 2 espèces en plus :
      
      * Mésange charbonnière : 4 phrases dans le lointain, mais bien distinctes, parfois un peu masquées
        par la Mésange des saules : à 1mn25s, 1mn31s, 1mn35s, 1mn40s,
      
      * Sitelle torchepôt : loin, mais assez clairs, des "tuit" scandés par 2 en général,
        à 1mn27.5s, 1mn31.5s, 1mn34.5s, 1mn37s, 1mn39s,  1mn44.5s.      
    """))

HTML(KNovClairiereArdenneBelge.lecteurHtml(urlDossierSons))

In [ ]:
KNovMesChaImitPouVelSitTor = quizz.DescripteurAnecdote(
  index='a',
  id='MesChaImitPouVelSitTor',
  titre="Une imitatrice surprenante ...",
  auteur="<a href=\"http://jpmeuret.free.fr/nature.html\" target=\"_blank\">Jean-Philippe Meuret</a>",
  licence="<a href=\"https://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr\" target=\"_blank\">CC BY-NC-SA</a>",
  fichierSon="180908-1559-EbreuilPlagnotsN-MesCha-imit-SitTor-ailes-fph-rb-a-c.vbr2.mp3", # nom fic. dans ./enregistrements
  dossierSons=urlDossierSons, dossierAttache=urlDossierAnnexes,
  texte="""
    On sait déjà que la Mésange charbonnière imite couramment les cris des autres mésanges,
    comme la Nonnette, et la Bleue, ou encore la "Longue queue" ...
    
    D'ailleurs ... les 3 premières de ces espèces sont tellement proches qu'elles s'hybrident parfois ;
    du coup, l'émission par une Charbonnière de cris typiques d'une Bleue étant tellement courante,
    voire systématique (obs. pers.), qu'on pourrait réellement se demander s'il s'agit d'imitations ;
    ces cris "de Bleue" (à nos oreilles peu adaptées) ne sont-ils pas simplement des cris hérités
    (sans grande mutation) de leur ancêtre commun dans l'évolution ?
    
    Mais ... si on n'a pas la preuve absolue que ces "cris de Bleue" ou de Nonnette
    ne font pas partie du vocabulaire propre à la Charbonnière, on a celle que l'espèce
    a de très grandes capacités pour en imiter d'autres ... et pas uniquement ses proches cousines les mésanges !
    
    L'exemple ci-dessus montre que la **Sitelle torchepot** elle-même n'y échappe pas ; et la Charbonnière
    que j'ai ici pris en flagrant délit n'a pas émis que ces 2 cris ... le temps que je réalise
    qu'il ne s'agissait pas d'une vraie sittelle (d'autres cris plus typiques de la Charbonnière m'ont
    mis la puce à l'oreille), que je me dise "Tiens, jamais entendu cette imitation" et que je me décide
    enfin à sortir mon enregistreur, le démarre, l'équipe de sa bonnette anti-vent, et appuie sur
    le bouton "Enregistrer" ... la faussaire commençait déjà à s'éloigner !
    
    Et les 2 autres exemples ci-dessous alourdissent encore le dossier de l'accusée, en mettant
    le **Pouillot véloce** juvénile (s'il vous plaît) à son tableau de chasse ...
    2 enregistrements captés à 10 mois d'intervalle, et en 2 endroits différents,
    mais seulement séparés de 500m (à vol de "contrefaisant") ... peut-être le même individu ?
    
    <table>
      <tr>
        <td>
          <a href="{{dossierSons}}/20170827-0827-StAgoulinEtangGiat-MesCha-imit-PouVel-GMoNoiCorNoiMerNoi-fph-rb.mp3" download>
            <img src="{{dossierAttache}}/20170827-0827-StAgoulinEtangGiat-MesCha-imit-PouVel-1ers-cris.jpg"/>
          </a>
          <p>Mésange charbonnière, 27/08/2017, Saint-Agoulin, Puy-de-Dôme, France</p>
          <audio controls>
            <source src="{{dossierSons}}/20170827-0827-StAgoulinEtangGiat-MesCha-imit-PouVel-GMoNoiCorNoiMerNoi-fph-rb.mp3" type="audio/mp3" preload="none"/>
          </audio>
          <p style="margin: 0 0 0 0">
            par <a href="http://jpmeuret.free.fr/nature.html" target="_blank">Jean-Philippe Meuret</a>,
            licence <a href="https://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr" target="_blank">CC BY-NC-SA</a>
          </p>
        </td>
        <td>
          <a href="https://www.xeno-canto.org/283062/download" download>
            <img src="{{dossierAttache}}/PouillotVeloce-cr-jvi-sweeo-16s-XC283062.jpg"/>
          </a>
          <p>Pouillot véloce, 27/09/2015, Arles, Bouches du Rhone, France</p>
          <audio controls>
            <source src="https://www.xeno-canto.org/283062/download" type="audio/mp3" preload="none"/>
          </audio>
          <p style="margin: 0 0 0 0">
            par <a href="https://www.xeno-canto.org/contributor/TJSRKPHQNP" target="_blank">Cedric Mroczko</a>,
            licence <a href="https://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr" target="_blank">CC BY-NC-SA</a>
          </p>
        </td>
      </tr>
    </table>

    Dans l'enregistrement ci-dessus à gauche, les cris "suîu" (ou "swee-o" comme disent les anglophones)
    de jeune pouillot en automne, à 0.5s, 4s, et 7s, imités par la charbonnière, sont bizarrement
    doublés 2 fois de suite, doublements qu'un vrai pouillot ne fait qu'occasionnellement ...
    
    La comparaison des sonagrammes de ces cris, Charbonnière à gauche, vrai Pouillot véloce à droite,
    montre une belle similarité ... au début des cris ; la différence de forme de la fin des cris,
    nette visuellement, ne l'est guère à l'oreille ... sans doute parce que tout cela va très vite,
    et aussi qu'au total, les cris originaux et imités ont la même durée.
    
    Notez quand même la roulade rêche typique de la Charbonnière à 2s et 5s ... les imitateurs se trahissent souvent
    à un moment ou à un autre, comme le dit 
    <a href="http://soundbirding.org/index.php/2018/04/18/some-food-for-thoughts-on-bird-imitations/"
     target="_blank">Stanislas Wroza</a>, et il "suffit" souvent d'être patient (enfin ici, pas besoin d'attendre).

    Notez également, en arrière plan, à 0.4s, 2.5s, 3s, 3.5s, et 6.5s au moins, des cris assez proches de ceux
    du "Pouillot charbonnier" (ou de la Mésange pouillotière" plutôt ?), mais sans doute plus courts :
    un Gobe-mouche noir de passage migratoire ... lui, pressé, émet plutôt des "vit" ;-)

    <table>
      <tr>
        <td>
          <audio controls>
            <source src="{{dossierSons}}/20180701-0851-StAgoulinEtangGiat-RouEff-avec-MesCha-imit-PouVel-fph-fpb-c.a5.mp3" type="audio/mp3" preload="none"/>
          </audio>
        </td>
        <td>
          <a href="{{dossierSons}}/20180701-0851-StAgoulinEtangGiat-RouEff-avec-MesCha-imit-PouVel-fph-fpb-c.a5.mp3" download>
            <img height="20" src="./fichiers/fa-download-solid.svg" alt="Télécharger"
                 title="Télécharger"/>
          </a>
        </td>
        <td>
          par <a href="http://jpmeuret.free.fr/nature.html" target="_blank">Jean-Philippe Meuret</a>
        </td>
        <td>
          licence <a href="https://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr" target="_blank">CC BY-NC-SA</a>
        </td>
      </tr>
    </table>

    En revanche, dans cet enregistrement (toujours une Charbonnière imitant les cris d'un jeune pouillot véloce),
    pas de "doublement" des cris ; mais une roulade typique (au début, juste avant 1s),
    qui peut aider à soupçonner l'entourloupe ...
    
    L'espèce qui chante en fond sonore continu, c'est une rousserole effarvate ... espèce qui a un point commun
    avec la Grive musicienne : si on essaie de décrire structurellement son chant, on peut arriver à qqch du genre :
    chant constitué d'une répétition infinie ... de 2 à 5 répétitions d'un motif parfois complexe ... choisi
    dans un répertoire parfois vaste et très varié :-)
    
    Bien sûr, le rythme est plus lent chez la musicienne, et les silences plus longs (!) ; et les motifs
    de l'Effarvate peuvent ne pas paraître si variés que cela tout compte fait ici ...
    
    Une dernière précision : les scènes sonores analysées ci-dessus ont été vécues en direct
    et tous les acteurs (sauf le Gobe-mouche noir) ont été identifiés visuellement ...
    sinon, sous la forme d'un quiz à l'aveugle, cela aurait été une autre "paire de manches",
    bien sûr ... une mission impossible ?
    """)

HTML(KNovMesChaImitPouVelSitTor.lecteurHtml(urlDossierSons))

In [ ]:
KRemerciements = \
"""
Grand merci également à <a href="https://ornithovaldallier.blogspot.com/" target="_blank">François Guélin</a>,
pour son bel enregistrement en forêt de conifères à Pessade, utilisé pour les quizz de Septembre.

Grand merci enfin à <a href="https://www.faune-auvergne.org/index.php?m_id=7&frmAuthor=32" target="_blank">
Romain Riols</a> pour ses splendides photos, en haut de page.
"""

In [ ]:
KQuizz = \
[quizz.DescripteurQuizz(id='Novembre', titre='Quiz de novembre',
                        intro="""
                          Trois quizz plus aérés cette fois, en milieux ouverts ou forestiers,
                          et un dernier plus "chargé" (quand même) dans une clairière habitée,
                          avec un invité d'honneur.
                        """,
                        publier=dict(lancement=True, indices2=True, reponse=True),
                        exercices=[KNovCoupeForetVienne, KNovBordEtangPologne,
                                   KNovMontadoVignesPortugal, KNovClairiereArdenneBelge],
                        anecdotes=[KNovMesChaImitPouVelSitTor]),
 quizz.DescripteurQuizz(id='Septembre', titre='Quiz de septembre',
                        intro="""
                          Deux ambiances forestières et une plus citadine ...
                        """,
                        publier=dict(lancement=True, indices2=True, reponse=True),
                        exercices=[KSeptPessadeForetConiferes, KSeptVienneForetPlaine, KSeptBerlinVille],
                        anecdotes=[KSeptConocephaleGracieux])
]

In [ ]:
# Génération de la page HTML.
nomFicCible = \
    quizz.buildHtmlPage(titre='Les Oiseaux à l\'Oreille : Quizzzzz 2018',
                        sousTitre='Et maintenant, prenez une feuille (enfin plutôt 2 ;-) !',
                        description='Quizz 2018 d\'application de la formation "Les Oiseaux à l\'Oreille"',
                        motsClef='chant, cri, oiseau, auvergne, fringilles, pics, mésanges',
                        preambule=KPreambule, quizz=KQuizz, etapes=KEtapes,
                        remerciements=KRemerciements, attributions=KAttributions,
                        dossierAttache=urlDossierAnnexes, dossierSons=urlDossierSons,
                        images=dict(imgTMat=[dict(img='FauTNo-m-RRiols-2013.jpg',
                                                  legend='Fauvette à tête noire mâle'),
                                             dict(img='MesNoi-RRiols-20130310.jpg',
                                                  legend='Mésange noire')]),
                        notebook='Quizz2018.ipynb', effort=32, prefixeFicCible='quizz-2018')

HTML("""<p>Quizz 2018 <a href='{nomFic}' target="_blank">{nomFic}</a>(nouvel onglet)</p>""".format(nomFic=nomFicCible))

In [ ]:
raise Exception("On s'arrête ici !")

In [ ]:
implib.reload(module=quizz)

In [ ]:
HTML(aideEnregistrementXC(nr=401097))